# Demonstration of MCMC non-linear regression with EMCEE and refnx

`refnx` is a package that can be used for non-linear regression (curvefitting). Here I demonstrate how it can be used to analyse Gaussian curve dataset, with Bayesian MCMC sampling of the posterior distributions of the parameters. This is a very robust way of estimating parameter uncertainties. I will also do the analysis with the `emcee` package for comparison

The first step is all the imports.

In [1]:
import numpy as np
import emcee
import corner
from scipy.optimize import leastsq
from refnx.analysis import CurveFitter
from refnx.dataset import Data1D
import refnx.analysis as ra
from matplotlib.pyplot import *
from lmfit import fit_report
%matplotlib inline
matplotlib.pyplot.rcParams['figure.figsize'] = (10.0, 6.0)
matplotlib.pyplot.rcParams['figure.dpi'] = 600

C:\Miniconda3\envs\testenv\lib\site-packages\PIL\Image.py:81: RuntimeWarning: The _imaging extension was built for another version of Python.
  RuntimeWarning
C:\Miniconda3\envs\testenv\lib\site-packages\PIL\Image.py:81: RuntimeWarning: The _imaging extension was built for another version of Python.
  RuntimeWarning
C:\Miniconda3\envs\testenv\lib\site-packages\PIL\Image.py:81: RuntimeWarning: The _imaging extension was built for another version of Python.
  RuntimeWarning
C:\Miniconda3\envs\testenv\lib\site-packages\PIL\Image.py:81: RuntimeWarning: The _imaging extension was built for another version of Python.
  RuntimeWarning


WARNING, Using slow reflectivity calculation


First step is to load some data in.

In [2]:
data = Data1D('gauss_data.txt')
errorbar(data.x, data.y, yerr=data.y_sd, fmt='.k')

FileNotFoundError: [Errno 2] No such file or directory: 'gauss_data.txt'

Define the fit functions. The first type is what you would use for `refnx.analysis.CurveFitter`, the second is a function that you can use with `emcee`.

In [ ]:
def gauss(x, p, *args):
    p0 = p.valuesdict()
    return p0['p0'] + p0['p1'] * np.exp(-((x - p0['p2']) / p0['p3'])**2)

def gauss2(x, p, *args):
    return p[0] + p[1] * np.exp(-((x - p[2]) / p[3])**2)

Set up initial parameter guesses and lower and upper bounds. The last step is to create an `lmfit.Parameters` instance for use with CurveFitter. The default parameter names created by `to_parameters` are 'p0', 'p1', ..., 'pn'.

In [ ]:
p0 = np.array([0.1, 20., 0.1, 0.1])
bounds_varying = np.array([(-1, 1), (0, 30), (-5., 5.), (0.001, 2)])
params = ra.to_parameters(p0, bounds=bounds_varying)

## Analyse with emcee

To start with we'll do the analysis with the `emcee` package. Then we'll repeat the analysis with `refnx.analysis.CurveFitter`. 

The following functions have to be defined for `emcee`. The log-likelihood, the uniform log-prior and the overall log-posterior probability.

In [ ]:
def residuals(theta):
    resid = (gauss2(data.x, theta) - data.y) / data.y_sd
    return resid
    
def lnlike(theta):
    # log likelihood
    return -0.5 * (np.sum(residuals(theta) ** 2))

def lnprior(theta):
    # uniform prior
    if (np.any(theta > bounds_varying[:, 1])
            or np.any(theta < bounds_varying[:, 0])):
        return -np.inf
    return 0

def lnpost(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta)

Lets fit the data with least squares first.

In [ ]:
result = leastsq(residuals, p0, full_output=True)
best_fit = result[0]
best_errors = np.sqrt(np.diag(result[1]))
for mean, std in zip(best_fit, best_errors):
    print("{:<12g} +/-  {:<10g}".format(mean, std))

Set up the walkers for `emcee`.

In [ ]:
ndim, nwalkers = 4, 100
pos = np.array([p0 * (1 + 1e-2 * np.random.randn(ndim))
    for i in range(nwalkers)])

Run the `emcee` sampler

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost)
a = sampler.run_mcmc(pos, 1000)

Discard 100 burn in steps for each walker and flatten the chain.

In [ ]:
chain = sampler.chain[:, 100:, :].reshape(-1, 4)

## Analyse with CurveFitter

Now we're going to do the analysis using a `refnx.analysis.CurveFitter` instance, it should be a lot simpler than the direct approach above. First setup the curvefitter.

In [ ]:
mini = CurveFitter(gauss, data, params)

First of all do a least-squares fit, to get a starting point for the sampling.

In [ ]:
res_leastsq = mini.fit()

Now do the MCMC sampling with CurveFitter instead. There are 100 walkers, we do 2000 steps on each walker. After the sampling discard the first 100 steps of each walker and take every 5th step

In [ ]:
# note that we initialise the emcee sampling with the output of the leastsq fit.
res_sampling = mini.emcee(nwalkers=100, steps=2000, burn=500, thin=10, params=res_leastsq.params)

The following plot shows the posterior distributions for each parameter

In [ ]:
b=corner.corner(res_sampling.flatchain)

But what about the fits, are they good?

In [ ]:
def pgen(parameters, flatchain, idx=None):
    # generator for all the different parameters from a flatchain.
    if idx is None:
        idx = range(np.size(flatchain, 0))
    for i in idx:
        vec = flatchain.iloc[i]
        for var_name in flatchain.columns:
            parameters[var_name].value = flatchain.iloc[i][var_name]
        yield parameters

In [ ]:
errorbar(data.x, data.y, yerr=data.y_sd, fmt=".")
for pars in pgen(res_sampling.params,
                 res_sampling.flatchain,
                 idx=np.random.choice(len(res_sampling.flatchain), size=500, replace=False)):
    plot(data.x, gauss(data.x, pars), color="k", alpha=0.05)
plot(data.x, gauss(data.x, res_sampling.params), color='r')

The following fit parameters are obtained. Lets compare them to the least squares output.

In [ ]:
print("Curvefitter.emcee")
print("-----------------")
print(fit_report(res_sampling.params))

print("\nleastsq")
print("-------")
print(fit_report(res_leastsq.params))